In [9]:
import pandas as pd
import numpy as np

In [16]:
Books_df = pd.read_csv('Books.csv',usecols=['ISBN', 'Book-Title'] , dtype={'ISBN': 'str', 'str': 'str'})

In [18]:
Ratings_df = pd.read_csv('Ratings.csv',usecols=['User-ID','ISBN', 'Book-Rating'] , dtype={'User-ID': 'int32','ISBN': 'str', 'Book-Rating': 'float'})

In [20]:
Books_df.head()

,ISBN,Book-Title
0,0195153448,Classical Mythology
1,0002005018,Clara Callan
2,0060973129,Decision in Normandy
3,0374157065,Flu: The Story of the Great Influenza Pandemic...
4,0393045218,The Mummies of Urumchi


In [21]:
Books_df.shape

(271360, 2)

In [22]:
Ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [23]:
Ratings_df.shape

(1149780, 3)

In [24]:
df = pd.merge(Ratings_df, Books_df, on = 'ISBN')

In [25]:
df

,User-ID,ISBN,Book-Rating,Book-Title
0,276725,034545104X,0.0,Flesh Tones: A Novel
1,2313,034545104X,5.0,Flesh Tones: A Novel
2,6543,034545104X,0.0,Flesh Tones: A Novel
3,8680,034545104X,5.0,Flesh Tones: A Novel
4,10314,034545104X,9.0,Flesh Tones: A Novel
...,...,...,...,...
1031131,276688,0517145553,0.0,Mostly Harmless
1031132,276688,1575660792,7.0,Gray Matter
1031133,276690,0590907301,0.0,Triplet Trouble and the Class Trip (Triplet Tr...
1031134,276704,0679752714,0.0,A Desert of Pure Feeling (Vintage Contemporaries)


In [35]:
combine_book_rating = df.dropna(axis = 0, subset = ['Book-Title'])
book_ratingCount = (combine_book_rating.
                     groupby(by=['Book-Title'])['Book-Rating']
                     .count().reset_index().
                     rename(columns= {'Book-Rating': 'totalRatingCount'})[['Book-Title','totalRatingCount']])
            
book_ratingCount

,Book-Title,totalRatingCount
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [36]:
rating_with_totalRatingCount = combine_book_rating.merge(book_ratingCount, left_on ='Book-Title', right_on='Book-Title', how='left')
rating_with_totalRatingCount.head()

,User-ID,ISBN,Book-Rating,Book-Title,totalRatingCount
0,276725,034545104X,0.0,Flesh Tones: A Novel,60
1,2313,034545104X,5.0,Flesh Tones: A Novel,60
2,6543,034545104X,0.0,Flesh Tones: A Novel,60
3,8680,034545104X,5.0,Flesh Tones: A Novel,60
4,10314,034545104X,9.0,Flesh Tones: A Novel,60


In [37]:
#pd.set_option('display, float_format', lambda x: '%.3f' % x)
#print(movie_ratingCount['totalRatingCount'].describe())

In [39]:
popularity_threshold = 50
rating_popular_Books = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_Books.head()

,User-ID,ISBN,Book-Rating,Book-Title,totalRatingCount
0,276725,034545104X,0.0,Flesh Tones: A Novel,60
1,2313,034545104X,5.0,Flesh Tones: A Novel,60
2,6543,034545104X,0.0,Flesh Tones: A Novel,60
3,8680,034545104X,5.0,Flesh Tones: A Novel,60
4,10314,034545104X,9.0,Flesh Tones: A Novel,60


In [40]:
rating_popular_Books.shape

(288740, 5)

In [42]:
#First lets create a Pivot Matrix

Books_features_df = rating_popular_Books.pivot_table(index='Book-Title', columns='ISBN',values='Book-Rating').fillna(0)
Books_features_df.head()

ISBN,0001047973,0001382608,0002220776,0002242591,0002261529,0002558122,0006170056,0006177379,0006385427,0006480764,000648302X,000648302x,000649840X,000649840x,000651202X,000651202x,0007107110,000711091X,0007110928,002026478X,0020442203,0020442505,0020697406,0020811853,002542730X,002542730x,0025706101,0026009102,0027110907,0027197107,0027587800,0027701301,0028604199,0030586232,0030615534,0048231398,0060001941,006000438X,006000438x,0060008024,...,8484505545,880781076X,8878242357,9500704447,9501513092,9504905501,9505112076,9506440298,9507424202,9604250620,9681500350,9875021245,B00001IVC7,B00001U0CP,B000051WXP,B00005BBW4,B00005NCS7,B00005Q8OL,B00006H3BO,B00006I4OX,B00006JO6O,B000078UH8,B00007CWQC,B00008WFXL,B00009ANY9,B00009EF82,B00009NDAN,B0000AA9JB,B0000C7BNG,B0000E63CJ,B0000T6KHI,B0000T6KIM,B0000VZEH8,B00011SOYM,B0001FZGBC,B0001FZGPI,B0001GMSV2,B0001GMSVW,B0001PIOX4,B000234N76
Book-Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Lighthouse Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010: Odyssey Two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
from scipy.sparse import csr_matrix

Books_features_df_matrix = csr_matrix(Books_features_df.values)

from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric= 'cosine' , algorithm = 'brute')
model_knn.fit(Books_features_df_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [47]:
Books_features_df.head()

ISBN,0001047973,0001382608,0002220776,0002242591,0002261529,0002558122,0006170056,0006177379,0006385427,0006480764,000648302X,000648302x,000649840X,000649840x,000651202X,000651202x,0007107110,000711091X,0007110928,002026478X,0020442203,0020442505,0020697406,0020811853,002542730X,002542730x,0025706101,0026009102,0027110907,0027197107,0027587800,0027701301,0028604199,0030586232,0030615534,0048231398,0060001941,006000438X,006000438x,0060008024,...,8484505545,880781076X,8878242357,9500704447,9501513092,9504905501,9505112076,9506440298,9507424202,9604250620,9681500350,9875021245,B00001IVC7,B00001U0CP,B000051WXP,B00005BBW4,B00005NCS7,B00005Q8OL,B00006H3BO,B00006I4OX,B00006JO6O,B000078UH8,B00007CWQC,B00008WFXL,B00009ANY9,B00009EF82,B00009NDAN,B0000AA9JB,B0000C7BNG,B0000E63CJ,B0000T6KHI,B0000T6KIM,B0000VZEH8,B00011SOYM,B0001FZGBC,B0001FZGPI,B0001GMSV2,B0001GMSVW,B0001PIOX4,B000234N76
Book-Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Lighthouse Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010: Odyssey Two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
query_index = np.random.choice(Books_features_df.shape[0]) 
print(query_index)
distances, indices = model_knn.kneighbors(Books_features_df.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 8)

2121


In [49]:
Books_features_df.head()

ISBN,0001047973,0001382608,0002220776,0002242591,0002261529,0002558122,0006170056,0006177379,0006385427,0006480764,000648302X,000648302x,000649840X,000649840x,000651202X,000651202x,0007107110,000711091X,0007110928,002026478X,0020442203,0020442505,0020697406,0020811853,002542730X,002542730x,0025706101,0026009102,0027110907,0027197107,0027587800,0027701301,0028604199,0030586232,0030615534,0048231398,0060001941,006000438X,006000438x,0060008024,...,8484505545,880781076X,8878242357,9500704447,9501513092,9504905501,9505112076,9506440298,9507424202,9604250620,9681500350,9875021245,B00001IVC7,B00001U0CP,B000051WXP,B00005BBW4,B00005NCS7,B00005Q8OL,B00006H3BO,B00006I4OX,B00006JO6O,B000078UH8,B00007CWQC,B00008WFXL,B00009ANY9,B00009EF82,B00009NDAN,B0000AA9JB,B0000C7BNG,B0000E63CJ,B0000T6KHI,B0000T6KIM,B0000VZEH8,B00011SOYM,B0001FZGBC,B0001FZGPI,B0001GMSV2,B0001GMSVW,B0001PIOX4,B000234N76
Book-Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Lighthouse Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010: Odyssey Two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendation for {0} :\n'.format(Books_features_df.index[query_index]))
    else:
        print('{0} {1}, with distance of {2}:' .format(i,Books_features_df.index[indices.flatten()[i]], distances.flatten()[i]))


Recommendation for The Rule of Four :

1 Summon the Keeper, with distance of 1.0:
2 Summer Sisters, with distance of 1.0:
3 Sun Also Rises, with distance of 1.0:
4 Superstitious, with distance of 1.0:
5 Summer Pleasures, with distance of 1.0:
6 Summer's End, with distance of 1.0:
7 Sunset in St. Tropez, with distance of 1.0:


In [ ]:
##Ahmat Baihaki - 17.52.0002